# Lesson 05 Augmented: Python API Walkthrough - USGS Earthquake Data

## Database Applications Development

### Overview
In this lesson, you'll work with real earthquake data from the United States Geological Survey (USGS). You'll learn to call a live API, parse geographic data, build a pandas DataFrame, and create interactive maps to visualize earthquake patterns around the world.

### Learning Objectives
- Call a real-world API from Python
- Parse GeoJSON data
- Build a pandas DataFrame
- Create an interactive map with folium

## Setup

Before we start, we need to import the libraries we'll use. If you get an error saying a module is not found, you may need to install it first with `pip install requests pandas folium`.

In [1]:
# -----------------------------------------------
# Setup: Import the libraries we need
# -----------------------------------------------

import requests              # lets us make HTTP requests to web APIs
import json                  # lets us work with JSON data
import pandas as pd          # gives us DataFrames for tabular data
from datetime import datetime  # lets us convert timestamps to dates

print("All imports successful.")

All imports successful.


## Part 1: The USGS Earthquake API

The USGS provides a free, public API for earthquake data at `https://earthquake.usgs.gov/fdsnws/event/1/query`. We can request earthquakes from a specific date range with a minimum magnitude. The API returns data in GeoJSON format, which is perfect for geographic data. First, we'll use the COUNT endpoint to see how many earthquakes match our search criteria before downloading all the data.

In [2]:
# -----------------------------------------------
# Part 1: Check how many earthquakes match our search
# -----------------------------------------------

# Hardcode our search parameters for 2025 earthquakes with magnitude 4.5+
start_date = '2025-01-01'
end_date = '2025-12-31'
min_magnitude = 4.5

print("Search Parameters:")
print(f"  Date range: {start_date} to {end_date}")
print(f"  Minimum magnitude: {min_magnitude}")
print()

# The USGS COUNT endpoint tells us how many results we'll get without downloading all the data
count_url = "https://earthquake.usgs.gov/fdsnws/event/1/count"

# Build the parameters dictionary - the API expects these as URL query parameters
params = {
    'format': 'geojson',
    'starttime': start_date,
    'endtime': end_date,
    'minmagnitude': min_magnitude
}

# Send the HTTP GET request to the USGS server
response = requests.get(count_url, params=params)

# Check the status code (200 means success)
print(f"HTTP Status Code: {response.status_code}")
print()

# Parse the JSON response and extract the count
count_data = response.json()
earthquake_count = count_data['count']

print(f"Number of M{min_magnitude}+ earthquakes: {earthquake_count}")

Search Parameters:
  Date range: 2025-01-01 to 2025-12-31
  Minimum magnitude: 4.5

HTTP Status Code: 200

Number of M4.5+ earthquakes: 8544


## Part 2: Fetching the Data

Now we'll download the actual earthquake data using the query endpoint.

In [3]:
# -----------------------------------------------
# Part 2: Download the actual earthquake data
# -----------------------------------------------

# The query endpoint returns the full data (not just a count)
query_url = "https://earthquake.usgs.gov/fdsnws/event/1/query"

# Same parameters we used for the count
params = {
    'format': 'geojson',
    'starttime': start_date,
    'endtime': end_date,
    'minmagnitude': min_magnitude
}

# Make the request (timeout after 30 seconds if server is slow)
print("Downloading earthquake data from USGS...")
response = requests.get(query_url, params=params, timeout=30)

# Check if the request worked (status code 200 means success)
if response.status_code == 200:
    print("Data downloaded successfully.")
else:
    print(f"Error: status code {response.status_code}")

# Parse the GeoJSON response into a Python dictionary
geojson_data = response.json()

# In GeoJSON, earthquake records are stored in a list called "features"
features = geojson_data['features']
print(f"Retrieved {len(features)} earthquake records.")
print()

# Let's look at the structure of ONE earthquake record to understand what data we have
print("Structure of the first earthquake record:")
print("=" * 60)
first_quake = features[0]

# Pretty-print the JSON (indent=2 makes it readable)
# Only show first 30 lines to keep it manageable
json_str = json.dumps(first_quake, indent=2)
lines = json_str.split('\n')
for line in lines[:30]:
    print(line)
if len(lines) > 30:
    print(f"... ({len(lines) - 30} more lines)")

Data downloaded successfully.
Retrieved 8544 earthquake records.

Structure of the first earthquake record:
{
  "type": "Feature",
  "properties": {
    "mag": 4.8,
    "place": "69 km E of Yamada, Japan",
    "time": 1767138696674,
    "updated": 1769048121040,
    "tz": null,
    "url": "https://earthquake.usgs.gov/earthquakes/eventpage/us7000rlm7",
    "detail": "https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=us7000rlm7&format=geojson",
    "felt": null,
    "cdi": null,
    "mmi": null,
    "alert": null,
    "status": "reviewed",
    "tsunami": 0,
    "sig": 354,
    "net": "us",
    "code": "7000rlm7",
    "ids": ",us7000rlm7,",
    "sources": ",us,",
    "types": ",origin,phase-data,",
    "nst": 65,
    "dmin": 1.853,
    "rms": 0.85,
    "gap": 128,
    "magType": "mb",
    "type": "earthquake",
    "title": "M 4.8 - 69 km E of Yamada, Japan"
  },
... (10 more lines)


## Key Insight: GeoJSON Format

GeoJSON is a standard way to represent geographic data. Each earthquake is a "Feature" with properties (magnitude, place, time) and geometry (coordinates). IMPORTANT: In GeoJSON, coordinates are [longitude, latitude, depth] - LONGITUDE FIRST, THEN LATITUDE. This is the opposite of how we usually think about it!

## Part 3: Building a DataFrame

Let's extract the information we need and organize it into a pandas DataFrame for analysis.

In [4]:
# -----------------------------------------------
# Part 3: Build a DataFrame from the earthquake data
# -----------------------------------------------

# Create an empty list to store earthquake records
earthquakes = []

# Loop through each earthquake in the GeoJSON features
for feature in features:
    # Get the properties dictionary (metadata about this earthquake)
    props = feature['properties']
    # Get the coordinates array [longitude, latitude, depth]
    coords = feature['geometry']['coordinates']
    
    # The USGS gives us time as Unix timestamp in milliseconds
    # We need to convert it to a readable datetime
    timestamp_ms = props['time']
    # Divide by 1000 to convert milliseconds to seconds for datetime conversion
    timestamp_sec = timestamp_ms / 1000
    # Use datetime.fromtimestamp to convert Unix timestamp to human-readable format
    earthquake_datetime = datetime.fromtimestamp(timestamp_sec)
    
    # Create a dictionary for this earthquake with all the fields we want
    record = {
        'magnitude': props['mag'],                    # Richter scale magnitude
        'place': props['place'],                      # Location description
        'datetime': earthquake_datetime,              # When it happened (converted from Unix time)
        'latitude': coords[1],                        # coords[1] = latitude (north/south)
        'longitude': coords[0],                       # coords[0] = longitude (east/west)
        'depth_km': coords[2],                        # coords[2] = depth in kilometers
    }
    # Add this earthquake record to our list
    earthquakes.append(record)

# Convert the list of dictionaries to a pandas DataFrame
df = pd.DataFrame(earthquakes)

# Sort by magnitude (largest earthquakes first), then reset index for clean numbering
df = df.sort_values('magnitude', ascending=False).reset_index(drop=True)

print(f"Created DataFrame with {len(df)} earthquakes")
print()

# Display the first 15 earthquakes (largest magnitudes)
print("Largest 15 earthquakes in the dataset:")
print("=" * 100)
# Show selected columns to keep output readable
display(df.head(15)[['magnitude', 'place', 'datetime', 'depth_km']])
print()

# Print summary statistics about the data
print("Summary Statistics:")
print(f"  Total earthquakes: {len(df)}")
print(f"  Magnitude range: {df['magnitude'].min():.1f} to {df['magnitude'].max():.1f}")
print(f"  Date range: {df['datetime'].min().date()} to {df['datetime'].max().date()}")
print(f"  Average depth: {df['depth_km'].mean():.1f} km")

Created DataFrame with 8544 earthquakes

Largest 15 earthquakes in the dataset:


,magnitude,place,datetime,depth_km
0,8.8,"2025 Kamchatka Peninsula, Russia Earthquake",2025-07-29 19:24:52.483,35.000
1,7.8,"140 km E of Petropavlovsk-Kamchatsky, Russia",2025-09-18 14:58:14.939,27.000
2,7.7,"2025 Mandalay, Burma (Myanmar) Earthquake",2025-03-28 02:20:52.715,10.000
3,7.6,Drake Passage,2025-10-10 16:29:20.075,5.639
4,7.6,"210 km SSW of George Town, Cayman Islands",2025-02-08 18:23:14.697,14.326
5,7.6,"2025 Aomori Prefecture, Japan Earthquake",2025-12-08 09:15:10.397,45.351
6,7.5,2025 Southern Drake Passage Earthquake,2025-08-21 22:16:18.246,10.000
7,7.4,2025 Drake Passage Earthquake,2025-05-02 08:58:26.014,10.000
8,7.4,"105 km E of Petropavlovsk-Kamchatsky, Russia",2025-09-12 22:37:56.761,58.000
9,7.4,"2025 Eastern Kamchatka, Russia Earthquake",2025-07-20 02:49:04.327,34.000



Summary Statistics:
  Total earthquakes: 8544
  Magnitude range: 4.5 to 8.8
  Date range: 2024-12-31 to 2025-12-30
  Average depth: 53.3 km


## Key Insight: Latitude and Longitude

Latitude measures position North or South (ranges from -90 to +90, where negative = South). Longitude measures position East or West (ranges from -180 to +180, where negative = West). In GeoJSON, coordinates come as [longitude, latitude] - EAST/WEST FIRST, THEN NORTH/SOUTH!

## Part 4: Exploring the Data

Now let's analyze the earthquake data to understand patterns.

In [5]:
# -----------------------------------------------
# Part 4: Explore the earthquake data
# -----------------------------------------------

# Get statistical summary of earthquake magnitudes
print("Magnitude Statistics:")
print("=" * 60)
# The describe() method gives us count, mean, std, min, 25%, 50%, 75%, max
print(df['magnitude'].describe())
print()

# Categorize earthquakes by magnitude range using pd.cut()
# This creates bins for different magnitude ranges
magnitude_bins = [0, 3, 4, 5, 6, 7, 10]
magnitude_labels = ['2.5-3.0', '3.0-4.0', '4.0-5.0', '5.0-6.0', '6.0-7.0', '7.0+']

# Add a new column with magnitude categories
df['magnitude_category'] = pd.cut(df['magnitude'], bins=magnitude_bins, labels=magnitude_labels, right=False)

# Count how many earthquakes are in each category
print("Earthquakes by Magnitude Range:")
print("=" * 60)
# value_counts() shows frequency, sort_index() orders them by range
magnitude_counts = df['magnitude_category'].value_counts().sort_index()
print(magnitude_counts)
print()

# Show the top 10 largest earthquakes with details
print("Top 10 Largest Earthquakes:")
print("=" * 100)
# Display first 10 rows with selected columns
display(df.head(10)[['magnitude', 'place', 'datetime', 'depth_km']])

Magnitude Statistics:
count    8544.000000
mean        4.810058
std         0.369414
min         4.500000
25%         4.500000
50%         4.700000
75%         4.900000
max         8.800000
Name: magnitude, dtype: float64

Earthquakes by Magnitude Range:
magnitude_category
2.5-3.0       0
3.0-4.0       0
4.0-5.0    6418
5.0-6.0    1982
6.0-7.0     128
7.0+         16
Name: count, dtype: int64

Top 10 Largest Earthquakes:


,magnitude,place,datetime,depth_km
0,8.8,"2025 Kamchatka Peninsula, Russia Earthquake",2025-07-29 19:24:52.483,35.000
1,7.8,"140 km E of Petropavlovsk-Kamchatsky, Russia",2025-09-18 14:58:14.939,27.000
2,7.7,"2025 Mandalay, Burma (Myanmar) Earthquake",2025-03-28 02:20:52.715,10.000
3,7.6,Drake Passage,2025-10-10 16:29:20.075,5.639
4,7.6,"210 km SSW of George Town, Cayman Islands",2025-02-08 18:23:14.697,14.326
5,7.6,"2025 Aomori Prefecture, Japan Earthquake",2025-12-08 09:15:10.397,45.351
6,7.5,2025 Southern Drake Passage Earthquake,2025-08-21 22:16:18.246,10.000
7,7.4,2025 Drake Passage Earthquake,2025-05-02 08:58:26.014,10.000
8,7.4,"105 km E of Petropavlovsk-Kamchatsky, Russia",2025-09-12 22:37:56.761,58.000
9,7.4,"2025 Eastern Kamchatka, Russia Earthquake",2025-07-20 02:49:04.327,34.000


## Key Insight: The Gutenberg-Richter Law

Seismologists discovered that for every 1.0 increase in magnitude, there are roughly 10 times fewer earthquakes. This is why we see thousands of M4.5-5.0 earthquakes per year, hundreds of M5.0-6.0, tens of M6.0-7.0, and very few M7.0+ earthquakes. This pattern holds globally!

In [9]:
! pip install folium

## Part 5: Mapping with Folium

Now we'll create an interactive map showing where these earthquakes occurred. Folium is a Python library that creates beautiful, interactive maps. If folium is not installed, use `pip install folium`.

In [10]:
# -----------------------------------------------
# Part 5: Import folium and test the import
# -----------------------------------------------

# Import folium for mapping
import folium

print("Folium imported successfully!")

Folium imported successfully!


In [11]:
# -----------------------------------------------
# Part 5: Create interactive earthquake map
# -----------------------------------------------

# Define a function to get the color based on earthquake magnitude
def get_color(magnitude):
    """Return a color based on earthquake magnitude."""
    # Green for smaller quakes
    if magnitude < 5.0:
        return 'green'
    # Orange for moderate quakes
    elif magnitude < 6.0:
        return 'orange'
    # Red for strong quakes
    elif magnitude < 7.0:
        return 'red'
    # Dark red for major quakes
    else:
        return 'darkred'

# Performance check: too many markers slow down the browser
# If we have more than 2000 earthquakes, only show the major ones
if len(df) > 2000:
    print(f"Note: {len(df)} earthquakes is a lot for an interactive map.")
    print("Showing only M5.0+ earthquakes to keep performance good.")
    # Filter to only show earthquakes with magnitude 5.0 or higher
    plot_df = df[df['magnitude'] >= 5.0]
    print(f"Plotting {len(plot_df)} earthquakes on the map.")
else:
    # If we have fewer than 2000, plot all of them
    plot_df = df
    print(f"Plotting all {len(plot_df)} earthquakes on the map.")

print()

# Create a world map centered at [latitude, longitude]
# [20, 0] is roughly the equator at the prime meridian
# zoom_start=2 gives us a zoomed-out world view
world_map = folium.Map(
    location=[20, 0],
    zoom_start=2,
    tiles='OpenStreetMap'
)

# Loop through each earthquake and add a marker to the map
for idx, row in plot_df.iterrows():
    # Get the color for this magnitude
    color = get_color(row['magnitude'])
    
    # Create HTML text for the popup (what appears when you click a marker)
    popup_text = f"""
    <b>{row['place']}</b><br>
    Magnitude: {row['magnitude']:.1f}<br>
    Date: {row['datetime'].strftime('%Y-%m-%d %H:%M:%S')}<br>
    Depth: {row['depth_km']:.1f} km<br>
    Location: ({row['latitude']:.2f}, {row['longitude']:.2f})
    """
    
    # Add a CircleMarker (colored circle) for this earthquake
    # CircleMarker creates a circle at the earthquake's coordinates
    folium.CircleMarker(
        # Center the circle at the earthquake's coordinates [latitude, longitude]
        location=[row['latitude'], row['longitude']],
        # Radius of the circle (in pixels on screen)
        radius=5,
        # The popup text that appears when you click the marker
        popup=folium.Popup(popup_text, max_width=300),
        # Color of the circle outline
        color=color,
        # Fill the circle with color
        fill=True,
        # Color used to fill the circle interior
        fillColor=color,
        # Opacity: 0 = transparent, 1 = fully solid
        fillOpacity=0.7,
        # Width of the circle outline in pixels
        weight=1
    ).add_to(world_map)  # Add this marker to our world_map object

print(f"Added {len(plot_df)} earthquake markers to the map")
print()
print("Click on any circle to see earthquake details!")
print()

# Save the map as an HTML file so it can be opened in a web browser
map_filename = 'earthquake_map.html'
world_map.save(map_filename)
print(f"Map saved as: {map_filename}")
print()

# Display the map in the notebook
world_map

Note: 8544 earthquakes is a lot for an interactive map.
Showing only M5.0+ earthquakes to keep performance good.
Plotting 2126 earthquakes on the map.

Added 2126 earthquake markers to the map

Click on any circle to see earthquake details!

Map saved as: earthquake_map.html



## Map Color Legend:

- Green circles: Magnitude < 5.0 (small earthquakes, rarely felt)
- Orange circles: Magnitude 5.0-5.9 (moderate, often felt)
- Red circles: Magnitude 6.0-6.9 (strong, significant damage possible)
- Dark red circles: Magnitude 7.0+ (major earthquakes, serious damage)

## Key Insight: The Ring of Fire

If you zoom out and look at the map, you'll notice earthquakes are NOT randomly scattered. They cluster along the edges of the Earth's tectonic plates, especially in a region called the Pacific Ring of Fire - a horseshoe-shaped area surrounding the Pacific Ocean. This is where the Earth's crust is most active, with plates constantly moving and colliding.

## Summary

You've completed a real-world data analysis project. Here's what you learned:

- Called a real-world API (USGS Earthquake Data) without authentication
- Parsed GeoJSON format geographic data
- Built and explored a pandas DataFrame
- Created an interactive map with folium

The earthquake_map.html file has been saved and contains an interactive map you can explore in any web browser. Zoom in, click on circles to see earthquake details, and examine the patterns of seismic activity around the world.

In [12]:
# -----------------------------------------------
# Completion message
# -----------------------------------------------

print("Lesson complete.")
print()
print("Your interactive earthquake map has been saved to: earthquake_map.html")
print("Open this file in a web browser to explore the map.")

Lesson complete.

Your interactive earthquake map has been saved to: earthquake_map.html
Open this file in a web browser to explore the map.
